In [1]:
%pip install -r requirements.txt

%pip install torch==2.1.1
%pip install --upgrade torchvision
%pip install --upgrade torchaudio

Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.1.1-cp39-cp39-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.3 MB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.17

In [2]:
# Import the libraries
import fitz
import os
from tqdm.auto import tqdm


In [3]:
pdf_path = './'
pdf_files = [f for f in os.listdir(pdf_path) if f.endswith(".pdf")]
print(pdf_files)

['Standford NLP.pdf', 'Attn_paper.pdf', 'Faiss.pdf', 'Gemini_paper.pdf', 'BART.pdf']


In [4]:
books_ = []

In [5]:
# Get Text content of the pdf
# For each pdf , get the pdf file contents 
# Store meta-data information about each pdf.

def format_text(text):
    text = text.strip("\n\n")
    text=text.replace("\n",'')
    return text

# # https://community.adobe.com/t5/acrobat-discussions/page-number-in-print-does-not-display-in-adobe-s-page-number-box/td-p/13781534
# Doesn't use Logical Page numbers. Use the normal Page numbers
# Logical causes issues during the rendering as you can't generalize to multiple pdfs

def get_text_content(pdf_text_, pdf_):
    doc = fitz.open(pdf_)
    print(len(doc))
    for page_num, page in enumerate(doc):
        # Extract the text content of the page
        text = page.get_text()
        text = format_text(text)
        name = pdf_.strip('.pdf')
        pdf_text_.append((text, page_num+1, name))
    return pdf_text_


In [6]:
books_ = []
books_ = [get_text_content([], pdf_) for pdf_ in pdf_files]

577
15
21
62
10


In [7]:
len(books_[1][7][0]) , books_[1][7]

(3107,
 ('Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on theEnglish-to-German and English-to-French newstest2014 tests at a fraction of the training cost.ModelBLEUTraining Cost (FLOPs)EN-DEEN-FREN-DEEN-FRByteNet [18]23.75Deep-Att + PosUnk [39]39.21.0 · 1020GNMT + RL [38]24.639.922.3 · 10191.4 · 1020ConvS2S [9]25.1640.469.6 · 10181.5 · 1020MoE [32]26.0340.562.0 · 10191.2 · 1020Deep-Att + PosUnk Ensemble [39]40.48.0 · 1020GNMT + RL Ensemble [38]26.3041.161.8 · 10201.1 · 1021ConvS2S Ensemble [9]26.3641.297.7 · 10191.2 · 1021Transformer (base model)27.338.13.3 · 1018Transformer (big)28.441.82.3 · 1019Residual DropoutWe apply dropout [33] to the output of each sub-layer, before it is added to thesub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and thepositional encodings in both the encoder and decoder stacks. For the base model, we use a rate ofPdrop = 0.1.Label SmoothingDuring training, we employed l

In [8]:
# Books_  -> [List[List[(text_content, page_num)]]

# Create a dictionary to store each pages documents into sentences

# Estimating that each token is 4 characters
# Page Info
def page_formatter(page):
    page_ = {}
    for pg in page:
        page_['doc_name'] = page[2]
        page_['text'] = page[0]
        page_['pg_num'] = page[1]
        page_['pg_num_chars'] = len(page[0])
        page_['pg_num_words'] = len(page[0].split(' '))
        page_['pg_num_sentences'] = len(page[0].split('. ')) # Since sentences usually begin with '. '
        page_['pg_num_tokens'] = page_['pg_num_chars'] / 4
    return page_
                                        
# Testing with a single page
# page_formatter(books_[1][6])

all_content_ = [[page_formatter(page) for page in books] for books in books_]


In [9]:
all_content_[3][3] # List[List[dictionaries]]


{'doc_name': 'Gemini_paper',
 'text': 'Gemini: A Family of Highly Capable Multimodal ModelsFigure 2 | Gemini supports interleaved sequences of text, image, audio, and video as inputs (illustratedby tokens of different colors in the input sequence). It can output responses with interleaved imageand text.tasks that require fine-grained understanding. In addition, Gemini can directly ingest audio signals at16kHz from Universal Speech Model (USM) (Zhang et al., 2023) features. This enables the model tocapture nuances that are typically lost when the audio is naively mapped to a text input (for example,see audio understanding demo on the website).Training the Gemini family of models required innovations in training algorithms, dataset, andinfrastructure. For the Pro model, the inherent scalability of our infrastructure and learning algorithmsenable us to complete pretraining in a matter of weeks, leveraging a fraction of the Ultra’s resources.The Nano series of models leverage additional ad

In [10]:
import pandas as pd
# df_p1_mean = pd.DataFrame(all_content_[0])['pg_num_tokens'].mean()
# # df_p2 = pd.DataFrame(all_content_[1])
# # df_p3 = pd.DataFrame(all_content_[2])
# # df_p4 = pd.DataFrame(all_content_[3])
# print(df_p1_mean)

# Calculate the mean number of sentences in each document.Then use sentence_splitter 
# to split the long sentences
def calc_mean(doc):
    doc_df = pd.DataFrame(doc)
    doc_mean = doc_df['pg_num_tokens'].mean()
    doc_sentences_mean = doc_df['pg_num_sentences'].mean()
    return doc_mean, doc_sentences_mean

for doc in all_content_:
    print("Token_mean , Sentences_mean",calc_mean(doc))

Token_mean , Sentences_mean (725.0064991334489, 32.85441941074524)
Token_mean , Sentences_mean (642.45, 17.0)
Token_mean , Sentences_mean (1178.2619047619048, 26.857142857142858)
Token_mean , Sentences_mean (581.2137096774194, 11.983870967741936)
Token_mean , Sentences_mean (1003.2, 18.7)


In [11]:
# Each sentence is long , the mean token size are above and won't fit the embedding model for tokenization
# Split sentences into smaller chunks

# Method to put all sentences in an array called sentences for making the splitting easier
def sentence_formatter_per_page(doc):
    for page in doc:
        page['sentences'] = [sentence for sentence in page['text'].split('. ')]
        

In [12]:
def sentence_formatter_all_books(books):
    for item in books:
        sentence_formatter_per_page(item)
    

In [13]:
sentence_formatter_all_books(all_content_)

In [14]:
all_content_[0][16:18]

[{'doc_name': 'Standford NLP',
  'text': '2.1•REGULAR EXPRESSIONS9any number of spaces! The star here applies only to the space ␣ that precedes it,not to the whole sequence. With the parentheses, we could write the expression/(Column␣[0-9]+␣*)*/ to match the word Column, followed by a number andoptional spaces, the whole pattern repeated zero or more times.This idea that one operator may take precedence over another, requiring us tosometimes use parentheses to specify what we mean, is formalized by the operatorprecedence hierarchy for regular expressions. The following table gives the orderoperatorprecedenceof RE operator precedence, from highest precedence to lowest precedence.Parenthesis()Counters* + ? {}Sequences and anchorsthe ˆmy end$Disjunction|Thus,becausecountershaveahigherprecedencethansequences,/the*/ matches theeeee but not thethe. Because sequences have a higher prece-dence than disjunction, /the|any/ matches the or any but not thany or theny.Patterns can be ambiguous in an

In [15]:
# Testing
def slice_sentences(page_sentences, overlap_size, slice_size):
    return [page_sentences[i:i + slice_size] for i in range(0, len(page_sentences) - slice_size + 1, slice_size - overlap_size)]


# Example usage:
page_sentences = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
overlap_size = 2
slice_size = 3
result = slice_sentences(page_sentences, overlap_size, slice_size)
print(result)

[[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10]]


In [16]:
# Based on this , splitting texts into size 8 since sizes are divisible by 6 (more or less)
# Token_mean , Sentences_mean (725.0064991334489, 32.85441941074524)
# Token_mean , Sentences_mean (642.45, 17.0)
# Token_mean , Sentences_mean (1178.2619047619048, 26.857142857142858)
# Token_mean , Sentences_mean (581.2137096774194, 11.983870967741936)
# Token_mean , Sentences_mean (1003.2, 18.7)

# Keep overlap of 1 sentence default for every 6 sentences
def slice_sentences(page_sentences, slice_size=5,overlap_size=1) :
    return [page_sentences[i:i + slice_size] for i in range(0, len(page_sentences) - slice_size + 1, slice_size - overlap_size)]

In [17]:

def all_content_sentence_splitter(all_content):
    for book in all_content:
        for page in tqdm(book):
            page["sentence_chunks"] = slice_sentences(page["sentences"],5,1)

            page["num_chunks"] = len(page["sentence_chunks"])

In [18]:
all_content_sentence_splitter(all_content_)

  0%|          | 0/577 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
import numpy as np
import random
random.sample(all_content_[2],k=3)

[{'doc_name': 'Faiss',
  'text': 'and a second level refinement of the residual (moreabout this in Section 5.1).4.5Binary indexesBinary quantization with symmetric distance com-putations is a pattern that has been commonlyused [Wang et al., 2015, Cao et al., 2017]. In this setup,distances are computed in the compressed domain asHamming distances. Equation (11) reduces to:n = argminn=1..N∥C(q) − Cn∥(17)where C(q), Cn ∈ {0, 1}d. Although binary quantizersare crude approximations for continuous domain dis-tances, the Hamming distances are integers in {0..d},that are fast to compute, do not require any specificcontext, and are easy to calibrate in practice.The Faiss IndexBinary indexes support additionand search directly from binary vectors. They offera compact representation and leverage optimized in-structions for distance computations.The simplest IndexBinaryFlat index performs ex-haustive search. Three options are offered for non-exhaustive search:• IndexBinaryIVF is a binary counterpa

In [20]:
pd.DataFrame(all_content_[0]).describe()

,pg_num,pg_num_chars,pg_num_words,pg_num_sentences,pg_num_tokens,num_chunks
count,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000
mean,289.000000,2900.025997,418.400347,32.854419,725.006499,7.608319
std,166.709828,1175.894326,155.640269,98.641975,293.973582,24.652779
min,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,145.000000,2349.000000,340.000000,10.000000,587.250000,2.000000
50%,289.000000,2773.000000,412.000000,13.000000,693.250000,3.000000
75%,433.000000,3232.000000,476.000000,17.000000,808.000000,4.000000
max,577.000000,11555.000000,1474.000000,994.000000,2888.750000,248.000000


In [21]:
pd.DataFrame(all_content_[1]).describe()


,pg_num,pg_num_chars,pg_num_words,pg_num_sentences,pg_num_tokens,num_chunks
count,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000
mean,8.000000,2569.800000,338.40000,17.000000,642.450000,3.600000
std,4.472136,1084.092853,166.50474,11.814035,271.023213,2.947154
min,1.000000,702.000000,45.00000,3.000000,175.500000,0.000000
25%,4.500000,2132.500000,304.50000,12.000000,533.125000,2.500000
50%,8.000000,3036.000000,383.00000,15.000000,759.000000,3.000000
75%,11.500000,3170.000000,459.00000,18.500000,792.500000,4.000000
max,15.000000,4203.000000,570.00000,43.000000,1050.750000,10.000000


In [22]:
pd.DataFrame(all_content_[2]).describe()


,pg_num,pg_num_chars,pg_num_words,pg_num_sentences,pg_num_tokens,num_chunks
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,11.000000,4713.047619,651.142857,26.857143,1178.261905,6.000000
std,6.204837,600.454701,98.317997,14.118377,150.113675,3.507136
min,1.000000,3133.000000,408.000000,13.000000,783.250000,3.000000
25%,6.000000,4422.000000,592.000000,19.000000,1105.500000,4.000000
50%,11.000000,4792.000000,651.000000,23.000000,1198.000000,5.000000
75%,16.000000,5049.000000,716.000000,24.000000,1262.250000,5.000000
max,21.000000,5733.000000,806.000000,63.000000,1433.250000,15.000000


In [23]:
pd.DataFrame(all_content_[3]).describe()

,pg_num,pg_num_chars,pg_num_words,pg_num_sentences,pg_num_tokens,num_chunks
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000
mean,31.500000,2324.854839,294.064516,11.983871,581.213710,2.419355
std,18.041619,1125.285987,150.230192,8.864170,281.321497,2.236423
min,1.000000,587.000000,87.000000,1.000000,146.750000,0.000000
25%,16.250000,1292.250000,125.500000,4.250000,323.062500,0.250000
50%,31.500000,2419.000000,329.000000,11.000000,604.750000,2.000000
75%,46.750000,3361.000000,413.750000,17.000000,840.250000,4.000000
max,62.000000,4105.000000,559.000000,36.000000,1026.250000,8.000000


In [24]:
# Split the sentence_chunks into it's own individual chunks to reduce size further.
import re
all_pdf_formatted = []
def final_chunk_dict(all_content, all_pdf_formatted):
    for doc in tqdm(all_content):
        for page in doc:
            for sentence_chunk in page["sentence_chunks"]:
                chunk_ = {}
                chunk_["pg_num"] = page["pg_num"]
                # Join sentences like a paragraph structure
                joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
                joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # spacing issue after join.
                chunk_["sentence_chunk"] = joined_sentence_chunk
                #write metadata
                chunk_["chunk_num_chars"] = len(joined_sentence_chunk)
                chunk_["chunk_num_tokens"] = len(joined_sentence_chunk) / 4
                chunk_["chunk_num_words"] = len([word for word in joined_sentence_chunk.split(" ")])
                chunk_["doc_name"] = page["doc_name"]
                all_pdf_formatted.append(chunk_)


In [25]:
final_chunk_dict(all_content_, all_pdf_formatted)

  0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
random.sample(all_pdf_formatted,k=1)

[{'pg_num': 335,
  'sentence_chunk': 'ﬁll back in correct slot values, resultingrelexicalizein “Au Midi has decent service”.15.4ChatbotsChatbots are systems that can carry on extended conversations with the goal ofchatbotmimicking the unstructured conversations or ‘chats’ characteristic of informal human-human interactionWhile early systems like ELIZA (Weizenbaum, 1966) or PARRY(Colby et al., 1971) had theoretical goals like testing theories of psychological coun-seling, for most of the last 50 years chatbots have been designed for entertainment. That changed with the recent rise of neural chatbots like ChatGPT, which incor-porate solutions to NLP tasks like question answering, writing tools, or machinetranslation into a conversational interfaceA conversation with ChatGPT is shownin Fig15.12In this section we describe neural chatbot architectures and datasets.[TBD]Figure 15.12A conversation with ChatGPT.15.4.1Training chatbotsDataChatbots are generally trained on a training set that in

In [27]:
all_pdf_formatted_df = pd.DataFrame(all_pdf_formatted)

In [28]:
all_pdf_formatted_df.describe()

,pg_num,chunk_num_chars,chunk_num_tokens,chunk_num_words
count,4761.000000,4761.000000,4761.000000,4761.000000
mean,273.468599,417.125814,104.281453,58.060912
std,245.631995,545.919045,136.479761,76.180003
min,1.000000,0.000000,0.000000,1.000000
25%,7.000000,27.000000,6.750000,3.000000
50%,257.000000,134.000000,33.500000,15.000000
75%,548.000000,758.000000,189.500000,109.000000
max,570.000000,7479.000000,1869.750000,458.000000


In [29]:
all_pdf_formatted_df = all_pdf_formatted_df[all_pdf_formatted_df['chunk_num_tokens'] >= 10]

In [30]:
all_pdf_formatted_df.tail(50)

,pg_num,sentence_chunk,chunk_num_chars,chunk_num_tokens,chunk_num_words,doc_name
4710,51,"It was my first time in the Big Apple, and I h...",351,87.75,70,Gemini_paper
4711,51,Iloved running around and playing fetchWe also...,321,80.25,55,Gemini_paper
4712,52,Gemini: A Family of Highly Capable Multimodal ...,704,176.00,107,Gemini_paper
4713,53,Gemini: A Family of Highly Capable Multimodal ...,347,86.75,61,Gemini_paper
4714,53,The roots of this equation are 𝑥1 = 5 and𝑥2 = ...,353,88.25,59,Gemini_paper
4715,53,The model shows good understanding of the task...,472,118.00,69,Gemini_paper
4716,54,Gemini: A Family of Highly Capable Multimodal ...,687,171.75,109,Gemini_paper
4717,57,Gemini: A Family of Highly Capable Multimodal ...,398,99.50,62,Gemini_paper
4718,60,Gemini: A Family of Highly Capable Multimodal ...,2132,533.00,345,Gemini_paper
4719,62,Gemini: A Family of Highly Capable Multimodal ...,627,156.75,94,Gemini_paper


In [31]:
all_pdf_formatted_df.describe()

,pg_num,chunk_num_chars,chunk_num_tokens,chunk_num_words
count,3384.000000,3384.000000,3384.000000,3384.000000
mean,362.370567,584.316785,146.079196,81.090130
std,217.691400,567.975995,141.993999,79.566382
min,1.000000,40.000000,10.000000,2.000000
25%,140.000000,120.000000,30.000000,13.000000
50%,482.000000,378.000000,94.500000,48.500000
75%,553.000000,960.250000,240.062500,139.000000
max,570.000000,7479.000000,1869.750000,458.000000


In [32]:
# Using only 1 step for reducing max_tokens
# Optimize later using recursion , if max_tokens_length still greater than 384
# Testing:

sample = {'pg_num': 406,
  'sentence_chunk': '398CHAPTER 18•DEPENDENCY PARSINGleading to a parse for the following example. Book me the morningﬂightiobjobjdetcompoundroot(18.7)Let’s consider the state of the conﬁguration at Step 2, after the word me has beenpushed onto the stack. StackWord ListRelations[root, book, me] [the, morning, ﬂight]The correct operator to apply here is RIGHTARC which assigns book as the head ofme and pops me from the stack resulting in the following conﬁguration. StackWord ListRelations[root, book] [the, morning, ﬂight] (book → me)After several subsequent applications of the SHIFT and LEFTARC operators, the con-ﬁguration in Step 6 looks like the following:StackWord ListRelations[root, book, the, morning, ﬂight][](book → me)Here, all the remaining words have been passed onto the stack and all that is leftto do is to apply the appropriate reduce operatorsIn the current conﬁguration, weemploy the LEFTARC operator resulting in the following state. StackWord ListRelations[root, book, the, ﬂight][](book → me)(morning ← ﬂight)At this point, the parse for this sentence consists of the following structure. Book me the morning ﬂightiobjcompound(18.8)There are several important things to note when examining sequences such asthe one in Figure 18.6First, the sequence given is not the only one that might leadto a reasonable parseIn general, there may be more than one path that leads to thesame result, and due to ambiguity, there may be other transition sequences that leadto different equally valid parses. Second, we are assuming that the oracle always provides the correct operatorat each point in the parse—an assumption that is unlikely to be true in practice. As a result, given the greedy nature of this algorithm, incorrect choices will lead toincorrect parses since the parser has no opportunity to go back and pursue alternativechoicesSection 18.2.4 will introduce several techniques that allow transition-basedapproaches to explore the search space more fully.',
  'chunk_num_chars': 1959,
  'chunk_num_tokens': 489.75,
  'chunk_num_words': 286,
  'doc_name': 'Standford NLP'}



length_chunk = len(sample['sentence_chunk'])
print(length_chunk//2)
# Split it into two

midpoint = length_chunk // 2
print(midpoint)
# Find the last period before the midpoint
split_point = sample['sentence_chunk'].rfind('. ', 0, midpoint)

# Split the text into two parts
first_half = sample['sentence_chunk'][:split_point+1]  # +1 to include the period
second_half = sample['sentence_chunk'][split_point+2:]  # +2 to skip the period and space

print(sample['sentence_chunk'])
print(len(first_half)/4)
print()
print(second_half)
            

979
979
398CHAPTER 18•DEPENDENCY PARSINGleading to a parse for the following example. Book me the morningﬂightiobjobjdetcompoundroot(18.7)Let’s consider the state of the conﬁguration at Step 2, after the word me has beenpushed onto the stack. StackWord ListRelations[root, book, me] [the, morning, ﬂight]The correct operator to apply here is RIGHTARC which assigns book as the head ofme and pops me from the stack resulting in the following conﬁguration. StackWord ListRelations[root, book] [the, morning, ﬂight] (book → me)After several subsequent applications of the SHIFT and LEFTARC operators, the con-ﬁguration in Step 6 looks like the following:StackWord ListRelations[root, book, the, morning, ﬂight][](book → me)Here, all the remaining words have been passed onto the stack and all that is leftto do is to apply the appropriate reduce operatorsIn the current conﬁguration, weemploy the LEFTARC operator resulting in the following state. StackWord ListRelations[root, book, the, ﬂight][](book 

In [33]:
# TODO: Optimize later using recursion , if max_tokens_length still greater than 384.
# For now just using 1/2 the length
# Rectifying the max_length tokens
def check_and_rectify_token_length(docs_df):
    # 384 is the max_token_length for some smaller models
    df_exceeding_length = docs_df[docs_df['chunk_num_tokens'] > 384]
    # Split further
    # Remove the df
    docs_df = docs_df[docs_df['chunk_num_tokens'] <= 384]
    # Convert them to dict
    doc_token_length_exceeded = df_exceeding_length.to_dict("records")
    # Run the splitter further. Split the chunks into two
    for doc in doc_token_length_exceeded:
        # Get sentencfe length
        doc_new_1 = {}
        doc_new_2 = {}
        length_chunk = len(doc['sentence_chunk'])
        # Split it into two
        # Assume `text` is your long text
        # Find the midpoint
        midpoint = length_chunk // 2

        # Find the last period before the midpoint
        split_point = doc['sentence_chunk'].rfind('. ', 0, midpoint)

        # Split the text into two parts
        first_half = doc['sentence_chunk'][:split_point+1]  # +1 to include the period
        second_half = doc['sentence_chunk'][split_point+2:]  # +2 to skip the period and space

        # Reclaculate all metadata except pg_num and doc_name            
        doc_new_1['pg_num'] = doc['pg_num']
        doc_new_2['pg_num'] = doc['pg_num']

        doc_new_1['doc_name'] = doc['doc_name']
        doc_new_2['doc_name'] = doc['doc_name']


        #write metadata
        doc_new_1['sentence_chunk'] = first_half
        doc_new_1["chunk_num_chars"] = len(first_half)
        doc_new_1["chunk_num_tokens"] = len(first_half) / 4
        doc_new_1["chunk_num_words"] = len([word for word in first_half.split(" ")])

        #write metadata
        doc_new_2['sentence_chunk'] = second_half
        doc_new_2["chunk_num_chars"] = len(first_half)
        doc_new_2["chunk_num_tokens"] = len(first_half) / 4
        doc_new_2["chunk_num_words"] = len([word for word in first_half.split(" ")])
        

        # Add it to the original dataframe

        docs_df = pd.concat([docs_df, pd.DataFrame([doc_new_1]), pd.DataFrame([doc_new_2])], ignore_index=True) 
    return docs_df

In [34]:
all_pdf_formatted_backup = all_pdf_formatted_df.to_dict('records')

In [35]:
pd.DataFrame(all_pdf_formatted_backup).describe()

,pg_num,chunk_num_chars,chunk_num_tokens,chunk_num_words
count,3384.000000,3384.000000,3384.000000,3384.000000
mean,362.370567,584.316785,146.079196,81.090130
std,217.691400,567.975995,141.993999,79.566382
min,1.000000,40.000000,10.000000,2.000000
25%,140.000000,120.000000,30.000000,13.000000
50%,482.000000,378.000000,94.500000,48.500000
75%,553.000000,960.250000,240.062500,139.000000
max,570.000000,7479.000000,1869.750000,458.000000


In [36]:
#Running rectified

all_pdf_formatted_df = check_and_rectify_token_length(all_pdf_formatted_df)

In [37]:
# Notice the increase in chunk_num_tokens count 
# max_ is less than 384
# min token 0
# 
all_pdf_formatted_df.describe()

,pg_num,chunk_num_chars,chunk_num_tokens,chunk_num_words
count,3583.000000,3583.000000,3583.000000,3583.000000
mean,355.840078,503.923807,125.980952,70.375384
std,217.237183,431.908522,107.977130,64.635809
min,1.000000,0.000000,0.000000,1.000000
25%,136.000000,117.000000,29.250000,13.000000
50%,456.000000,355.000000,88.750000,46.000000
75%,552.000000,850.000000,212.500000,123.000000
max,570.000000,1534.000000,383.500000,287.000000


In [38]:
pd.DataFrame(all_pdf_formatted_backup).describe()

,pg_num,chunk_num_chars,chunk_num_tokens,chunk_num_words
count,3384.000000,3384.000000,3384.000000,3384.000000
mean,362.370567,584.316785,146.079196,81.090130
std,217.691400,567.975995,141.993999,79.566382
min,1.000000,40.000000,10.000000,2.000000
25%,140.000000,120.000000,30.000000,13.000000
50%,482.000000,378.000000,94.500000,48.500000
75%,553.000000,960.250000,240.062500,139.000000
max,570.000000,7479.000000,1869.750000,458.000000


In [39]:
all_pdf_formatted_dict = all_pdf_formatted_df.to_dict('records')

In [40]:
random.sample(all_pdf_formatted_dict,k=2)

[{'pg_num': 209,
  'sentence_chunk': '',
  'chunk_num_chars': 0,
  'chunk_num_tokens': 0.0,
  'chunk_num_words': 1,
  'doc_name': 'Standford NLP'},
 {'pg_num': 414,
  'sentence_chunk': 'Thus a maximum spanning tree of G emanating from theROOT is the optimal dependency parse for the sentence. A directed graph for the example Book that ﬂight is shown in Fig18.11, with themaximum spanning tree corresponding to the desired parse shown in blueFor easeof exposition, we’ll describe here the algorithm for unlabeled dependency parsing.rootBookthatﬂight1244568757Figure 18.11Initial rooted, directed graph for Book that ﬂight. Before describing the algorithm it’s useful to consider two intuitions about di-rected graphs and their spanning treesThe ﬁrst intuition begins with the fact thatevery vertex in a spanning tree has exactly one incoming edgeIt follows from thisthat every connected component of a spanning tree (i.e., every set of vertices thatare linked to each other by paths over edges) will 

## Embedding and the retriever
- Generate the embeddings for query and generate embeddings for the documents
- Store the document embeddings in the vector store : Use FAISS
- Serialize the vector store for later use
- Test it later using load

In [41]:
# Generate the embeddings for query and generate embeddings for the documents
# Store the document embeddings in the vector store : Use FAISS
# Serialize the vector store for later use
# Test it later using load
# Create the Retriever function to return the scores and indices. Create the print function to print relevant score, docs
# Import the llm for generation. Use bitsandBytes config. Create prompt template
# Add the context to prompt along with query format.
# apply the chat template 
# Generate the answer using LLM

# Sample testing
from sentence_transformers import SentenceTransformer, util

embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


query_sample = "What does Darryl's friends do?"
answer_samples = ["Darryl pursued his Masters in Computer Science", "Darryl's friends study Information systems","Studying Generative AI is a boost to the mind"]

answer_embeddings = embed_model.encode(answer_samples)
query_embedding = embed_model.encode(query_sample)

scores = util.cos_sim(query_embedding, answer_embeddings)

In [42]:
def sentence_similarity(scores, query_embedding, answer_embeddings, query, answers):
    print(answer_embeddings.shape)
    score = util.cos_sim(query_embedding, answer_embeddings)
    score,idx = score, score.argmax(-1)
    print(f"QUERY: {query}")
    print(f"ANSWER: {answers[idx]}")
    

In [43]:
sentence_similarity(scores, query_embedding, answer_embeddings, query_sample, answer_samples)

(3, 768)
QUERY: What does Darryl's friends do?
ANSWER: Darryl's friends study Information systems


In [44]:
%%time
### Encoding and Retriever

# Change embed_model to 'cuda'
embed_model.to('cuda')

# Returns
# By default, a 2d numpy array with shape [num_inputs, output_dimension] is returned.
# If only one string input is provided, then the output is a 1d array with shape [output_dimension]. 
# If convert_to_tensor, a torch Tensor is returned instead.

#Utility for encoding chunks of documents
def encode_sentences(documents):
    for doc in tqdm(documents):
        doc["embeddings"] = embed_model.encode(doc["sentence_chunk"], convert_to_tensor=True)
        
encode_sentences(all_pdf_formatted_dict)       

  0%|          | 0/3583 [00:00<?, ?it/s]

CPU times: user 4min 10s, sys: 2.07 s, total: 4min 12s
Wall time: 1min 16s


In [45]:
import time

In [46]:
%%time
# Using batched mode
# sentence_chunks = [doc['sentence_chunk'] for doc in all_pdf_formatted_dict]
# print(len(sentence_chunks))
# print(len(all_pdf_formatted_dict))
# print(sentence_chunks[3341])
embed_model.to('cuda')


def encode_sentences_batched(documents):
    sentence_chunks = [doc['sentence_chunk'] for doc in documents]
    print(len(sentence_chunks))
    print(len(all_pdf_formatted_dict))
    batched_sentence_embeddings = embed_model.encode(sentence_chunks, batch_size=15, convert_to_tensor=True)
    return batched_sentence_embeddings

batched_sentence_embeddings = encode_sentences_batched(all_pdf_formatted_dict)

3583
3583
CPU times: user 36.3 s, sys: 8.24 s, total: 44.6 s
Wall time: 27.8 s


In [47]:
print("Embeddings using batch_size=15:",batched_sentence_embeddings.shape)
print("Embeddings without batched ",len(all_pdf_formatted_dict) , all_pdf_formatted_dict[0]['embeddings'].shape)

Embeddings using batch_size=15: torch.Size([3583, 768])
Embeddings without batched  3583 torch.Size([768])


In [48]:
print(batched_sentence_embeddings[0].type())
print(batched_sentence_embeddings[0].dtype)

torch.cuda.FloatTensor
torch.float32


In [49]:
type(all_pdf_formatted_dict[0]['embeddings'])

torch.Tensor

In [50]:
# Storing embeddings in vectorStore
# Experimental: Can use Pinecone, Milvus, AstraDB, MongoDB etc
# Using FAISS as the vector store
%pip install  faiss-cpu==1.8.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [51]:
import faiss

dimension = embed_model[1].word_embedding_dimension
print(f"Embed_model",embed_model)
print(f"Dimension : {dimension}")

# def get_dimension(embedding_model):
    
index = faiss.IndexFlatL2(dimension)   

# build the index, d=size of vectors 
# assume xb contains a n-by-d numpy matrix of type float32
# index.add(xb)                          
# add vectors to the index

# TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

# Sol: https://stackoverflow.com/questions/53467215/convert-pytorch-cuda-tensor-to-numpy-array


def convert_to_np(embeddings):
    batched_embeddings_np = embeddings.detach().cpu().numpy()
    return batched_embeddings_np

# type(batched_embeddings_np)

batched_sentence_embeddings_np = convert_to_np(batched_sentence_embeddings)
print(f"Embeddings shape : {batched_sentence_embeddings_np.shape} and type {type(batched_sentence_embeddings_np)}")
print("Adding FAISS index")
index.add(batched_sentence_embeddings_np)
print(f"Added total indexes: {index.ntotal}")


Embed_model SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Dimension : 768
Embeddings shape : (3583, 768) and type <class 'numpy.ndarray'>
Adding FAISS index
Added total indexes: 3583


In [52]:
# Serialize the index
faiss.write_index(index, "index_embeddings.bin")
index2 = faiss.read_index("index_embeddings.bin")  # index2 is identical to index



In [53]:
# Database mapping of sentence-chunks and embeddings
# For now use a csv to store the embeddings and text chunk mapping
# For scaling: Database required : use mongodb or sql 
data_csv = pd.DataFrame(all_pdf_formatted_dict).to_csv("data_.csv", index=False)

In [54]:
#Testing search
query = "How was Gemini invented?"
embed_model.to('cuda')

query_test = embed_model.encode(query)
print(query_test.shape, type(query_test))
distance, ids = index2.search(np.expand_dims(query_test, axis=0), 5)

# second argument specifies the number of closest documents you want to retrieve
#Matrix distance is the matrix of squared distances. It has the same shape as I and indicates for each result vector at the query’s squared Euclidean distance.

first_doc_id = ids[0][0]
original_doc = all_pdf_formatted_dict[first_doc_id]

print(f"QUERY:{query} \n\n: ANS:{original_doc} , DISTANCE: {distance[0][0]}")


(768,) <class 'numpy.ndarray'>
QUERY:How was Gemini invented? 

: ANS:{'pg_num': 57, 'sentence_chunk': 'Gemini: A Family of Highly Capable Multimodal Models9.4.1Reasoning and code generationPromptCreate a web app called "Opossum Search":1Every time you make a search query, it should redirect you to a google search with the samequery, but the word opossum before it.2It should be visually similar to Google search,3Instead of the google logo, it should have a picture of an opossum from the internet.4', 'chunk_num_chars': 398, 'chunk_num_tokens': 99.5, 'chunk_num_words': 62, 'doc_name': 'Gemini_paper', 'embeddings': tensor([ 5.7879e-02,  7.4607e-02, -3.0555e-02, -3.1337e-02, -1.6591e-02,
        -2.1249e-02,  2.0056e-02,  1.6419e-02, -4.9016e-02, -4.0856e-02,
        -2.3328e-03, -6.0466e-03, -4.7642e-02,  1.1492e-01, -5.0542e-03,
        -1.0370e-01,  3.7736e-02, -4.6329e-03, -5.8520e-02, -2.2963e-02,
        -3.4077e-02, -1.3310e-02, -5.1538e-03,  1.1935e-02,  6.6984e-03,
        -5.1249

In [55]:
# Making the output pretty
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text , wrap_length)
    print(wrapped_text)

In [147]:
from time import perf_counter as timer
# Function for retriver
def retrieve_docs(query, doc_embeddings, all_docs_array, model, faiss_index,n_return_docs=5, print_time=True, print_docs=True):
    """ Embed the query , and return the top docs distance, indices"""
    model = model.to('cuda')
    # Query embedding
    query_embedding = model.encode(query)
    
    # Reshaping since faiss expects nxd ndarry format
    query_embedding = np.expand_dims(query_embedding, axis=0)
    
    # start time
    time_strt = timer()
    distances, retrieved_ids = faiss_index.search(query_embedding, n_return_docs)
    time_end = timer()
    
    if print_time:
        print(f"[INFO] Time taken to get search on {len(doc_embeddings)} embeddings: {time_end-time_strt:.5f} seconds")
    
    if print_docs:
        print(f"QUERY: {query} \n")
        print("Retrieved Docs: \n")
        for doc_num,doc_id in enumerate(retrieved_ids[0]):
            print(f"DOC {doc_num}")
            print(f"\n {all_pdf_formatted_dict[doc_id]['sentence_chunk']}")
            print("\n")
    return retrieved_ids[0]

In [148]:
retrieve_docs("Explain recurrent nueral networks", batched_sentence_embeddings_np,all_pdf_formatted_dict, embed_model, \
              index2, n_return_docs=6, print_time=True, print_docs=True)

[INFO] Time taken to get search on 3583 embeddings: 0.00163 seconds
QUERY: Explain recurrent nueral networks 

Retrieved Docs: 

DOC 0

 188CHAPTER 9•RNNS AND LSTMSworksThese networks are useful in their own right and serve as the basis for morecomplex approaches like the Long Short-Term Memory (LSTM) networks discussedlater in this chapterIn this chapter when we use the term RNN we’ll be referring tothese simpler more constrained networks (although you will often see the term RNNto mean any net with recurrent properties including LSTMs).xthtytFigure 9.1Simple recurrent neural network after Elman (1990)The hidden layer includesa recurrent connection as part of its inputThat is, the activation value of the hidden layerdepends on the current input as well as the activation value of the hidden layer from theprevious time step. Fig


DOC 1

 202CHAPTER 9•RNNS AND LSTMS+xtht-1cthtcthtct-1ht-1xttanh+σtanhσσ+++igfo⦿⦿⦿LSTMct-1Figure 9.13A single LSTM unit displayed as a computation graphThe in

array([ 572,  601, 1824,  571,  574,  578])

# Generator 
1. Figure out the compute requirements. Quantize model according to your GPU RAM. 
2. Calculate parameters of your llm and see if there is enough RAM to load the model.
3. Instruction tuned version. 

In [58]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 8 GB


In [59]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Mistral AI LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Mistral 7B in 4-bit precision.")
    use_quantization_config = True 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Mistral 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False 
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Mistral 7B in 4-bit or float16 precision.")
    use_quantization_config = False 
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 8 | Recommended model: Mistral 7B in 4-bit precision.
use_quantization_config set to: True
model_id set to: google/gemma-2b-it


In [60]:
# %pip install torch==2.1.1
# %pip install --upgrade torchvision
# %pip install --upgrade torchaudio

In [61]:
# Using mistral-ai

import torch

# Check if PyTorch is installed
try:
    torch_version = torch.__version__
    print(f"PyTorch version: {torch_version}")
except ImportError:
    print("PyTorch is not installed.")


PyTorch version: 2.2.2+cu121


In [64]:
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

device = "cuda" # the device to load the model onto


# https://github.com/huggingface/transformers/blob/v4.39.3/src/transformers/utils/quantization_config.py#L182
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16, bnb_4bit_quant_type="nf4")

# Check  scaled dot product attn + flash_attn


# Model instantiated with quantization config
llm_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", \
                                                   quantization_config=quant_config,
                                                 low_cpu_mem_usage=True)

# Model tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, )


# llm_model.to('cuda') -> Not available for quantized version.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

You shouldn't move a model that is dispatched using accelerate hooks.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [65]:
llm_model
# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

# model_inputs = encodeds.to(device)
# model.to(device)

# generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [66]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Apr  3 19:32:45 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     Off  | 00000000:00:05.0 Off |                  N/A |
| 30%   32C    P8     5W / 125W |   6361MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [69]:
def get_model_params(model):
    return sum([torch.numel(param) for param in model.parameters()])

get_model_params(llm_model)

3752071168

In [70]:
# Define model memory size on our machine
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)
# Minimum required is 4GB, but due to gradients etc, would require about 7 GB.

{'model_mem_bytes': 4551360512, 'model_mem_mb': 4340.52, 'model_mem_gb': 4.24}

In [81]:
print(torch.cuda.current_device())
print(torch.cuda.memory_allocated(device=0))
print(torch.cuda.get_device_properties(0))

0
5745254400
_CudaDeviceProperties(name='Quadro RTX 4000', major=7, minor=5, total_memory=7974MB, multi_processor_count=36)


In [72]:
torch.cuda.is_available()

True

In [73]:
torch.cuda.device_count()

1

In [98]:

query = "What are the differences between BERT and BART models?"

print(f" [QUERY] : {query}")
# Chat template
chat_tempalte = [
    {"role": "user", 
     "content": query
    }]

# Instruction format
# In order to leverage instruction fine-tuning, your prompt should be surrounded by [INST] and [/INST] tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

# E.g.

# text = "<s>[INST] What is your favourite condiment? [/INST]"
# "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
# "[INST] Do you have mayonnaise recipes? [/INST]"
# 

# apply_chat_template(
#         self,
#         conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], "Conversation"],
#         chat_template: Optional[str] = None,
#         add_generation_prompt: bool = False,
#         tokenize: bool = True,
#         padding: bool = False,
#         truncation: bool = False,
#         max_length: Optional[int] = None,
#         return_tensors: Optional[Union[str, TensorType]] = None,
#         return_dict: bool = False,
#         tokenizer_kwargs: Optional[Dict[str, Any]] = None,
#         **kwargs,
#     ) -> Union[str, List[int], List[str], List[List[int]], BatchEncoding]:

prompt = tokenizer.apply_chat_template(conversation=chat_tempalte, tokenize=False, add_generation_prompt=True, return_tensors="pt")

print(f"Formatter Template : {prompt}")



 [QUERY] : What are the differences between BERT and BART models?
Formatter Template : <s>[INST] What are the differences between BERT and BART models? [/INST]


In [99]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [100]:
input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
input_ids

# Generate output from llm
output_tokens = llm_model.generate(**input_ids, max_new_tokens=256)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [102]:
# Decoding output tokens
print(f"Output [tokens]: {output_tokens[0]}\n")

tokenizer.decode(output_tokens[0])

Output [tokens]: tensor([    1,     1,   733, 16289, 28793,  1824,   460,   272, 11090,  1444,
          365,  4538,   304,   365,  4520,  4994, 28804,   733, 28748, 16289,
        28793,   365,  4538,   325, 28760,   313,  7819,   282, 17611,  5722,
        17891,   697,   477,  4335,   674,   404, 28731,   304,   365,  4520,
          325, 19064,  3521, 16107, 28742, 28713,  4745, 28709,  2979, 12191,
        20787,   354,  7379, 28733,   532, 28733,  1874,  8871,   288, 28731,
          460,  1560,  1665, 28733,  1009, 28733,  1237, 28733,   444,  4994,
          297,   272,  1834,   302, 16725, 15589, 10705,   288,   325, 28759,
        11661,   557,   562,   590,  7031,  1581, 10700,   304,   506,  9494,
         7821,  1890, 11090, 28723,    13,    13, 28760,  4538,   349,   264,
          710, 28733, 28247,  5516,   263,  2229,   369,   541,   347,  4433,
        28733, 28707,   370,   286,   354,  4118,   418, 11661,  9796,  1259,
          390, 16776, 28725,  5160,  9040, 1382

"<s><s> [INST] What are the differences between BERT and BART models? [/INST] BERT (Bidirectional Encoder Representations from Transformers) and BART (Facebook AI's Denoising Autoencoder for Text-to-Text Modeling) are both state-of-the-art models in the field of Natural Language Processing (NLP), but they serve different purposes and have distinct architectural differences.\n\nBERT is a pre-trained transformer model that can be fine-tuned for various NLP tasks such as classification, named entity recognition, question answering, and text generation, among others. It is a bidirectional model, meaning it considers the context from both left and right contexts when making predictions. BERT uses a masked language modeling objective during pre-training, where it predicts missing words in a sentence based on the context of the surrounding words.\n\nBART, on the other hand, is a denoising autoencoder that can be fine-tuned for text generation tasks. It is an autoregressive model, meaning it g

In [286]:
sample_questions=['What is Faiss? Summarize the working of Faiss.',
'What are the different types of quantizers used in Faiss?',
'What are Additive quantizers?',
'What are Multi-codebook quantizers?',
'Can you describe the pretraining process of BART and its architecture?',
'Could you provide examples of where BART has been particularly effective?',
'Are there any known challenges or limitations when using BART for certain tasks?',
'What are the main principles behind the GEMINI framework?',
'How does GEMINI address the challenges of interpretability in machine learning models?',
'Can you explain the process of model extraction and perturbation used in GEMINI?',
'What are some potential applications of GEMINI in real-world scenarios?',
'What is the model architecture of Gemini?',
'What is the significance of attention mechanisms in neural networks, as discussed in the Attention paper?',
'How does the Attention paper propose to improve attention mechanisms in deep learning models?',
'How do the proposed improvements of multi-head attention compare with existing single head attention mechanisms?',
'What are recurrent nueral networks ? Are they time based deep learning models ?',
'When do you see butterflies in the sky?',
'Is there food in the kitchen?']


import random
query = random.sample(sample_questions,k=1)
print(f"Query:  {query[0]}")



retrieve_docs(query[0], batched_sentence_embeddings_np,all_pdf_formatted_dict, embed_model, index2, n_return_docs=6, print_time=True)

Query:  What is the significance of attention mechanisms in neural networks, as discussed in the Attention paper?
[INFO] Time taken to get search on 3583 embeddings: 0.00190 seconds
QUERY: What is the significance of attention mechanisms in neural networks, as discussed in the Attention paper? 

Retrieved Docs: 

DOC 0

 The attention mechanism is a solution to the bottleneck problem, a way ofattentionmechanismallowing the decoder to get information from all the hidden states of the encoder,not just the last hidden state. In the attention mechanism, as in the vanilla encoder-decoder model, the contextvector c is a single vector that is a function of the hidden states of the encoder, thatis, c = f(he1 ...hen)Because the number of hidden states varies with the size ofthe input, we can’t use the entire set of encoder hidden state vectors directly as thecontext for the decoder. The idea of attention is instead to create the single ﬁxed-length vector c by takinga weighted sum of all the enc

array([3296,  628,  636, 3511,  621, 2891])

In [359]:
# Augmentation of query + retrieved_docs
# Pass to llm


# Using Zero shot learning
def data_augmenter(query, documents):
    context = "\n - " + "\n - ".join([doc["sentence_chunk"] for doc in documents])
    # base_prompt = """Based on the following context documents, please answer the query.
    # Think about the question and use the context to answer the query.
    # Do not return the thinking, only return the answer.
    # Make sure to provide detailed explainations.If you think there is not enough information in the context documents, say that you do not have enough information.
    # Here are a few examples for your referance. Use it as a reference for the answer.
    # \nExample 1:
    # Query: What is the best resource to learn generative AI ?
    # Answer: Generative AI is a new and emerging field. The best way to learn is by practically implementing projects using HuggingFace transformers, LangChain and other available tools.
    # \nExample 2:
    # Query: What is asdsdfvxcvsd ?
    # Answer: I am sorry but I do not understand the question. Kindly re-correct the question or rephrase it to a meaningful ask.
    # \nNow use the following context below to answer the user query: 
    # {context}
    # \nRelevant passages: <extract relevant passages from the context here>
    # User Query: {query}
    # Answer:"""
    base_prompt = """<s>[INST] Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:
[/INST]"""

    base_prompt = base_prompt.format(context=context, query=query)
    # Chat Template
    chat_template = [
                    {"role": "user", 
                     "content": base_prompt
                    }
                  ]
    
    prompt = tokenizer.apply_chat_template(conversation=chat_template, tokenize=False, add_generation_prompt=True)

    
    return prompt

# Choose a random query
query = random.sample(sample_questions,k=1)
print(f"Query: {query[0]}")
# Get relevant context
doc_ids = retrieve_docs(*query, batched_sentence_embeddings_np,all_pdf_formatted_dict, embed_model,\
                        index2, n_return_docs=5, print_time=False, print_docs=False)

# Context documents
context_docs = [all_pdf_formatted_dict[i] for i in doc_ids]
# Format prompt
prompt = data_augmenter(*query, documents=context_docs)
print(prompt)


Query: Can you explain the process of model extraction and perturbation used in GEMINI?
<s>[INST] <s>[INST] Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Qu

In [361]:
#tokenizer.pad_token_id = tokenizer.eos_token_id
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generating outputs using llm mistral-instruct now
outputs = llm_model.generate(**input_ids, temperature=0.7,do_sample=True, max_new_tokens=256)

generated_output = tokenizer.decode(outputs[0])

print(f"Query: {query[0]}")
print(f"Answer:{generated_output.replace(prompt, '')}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Can you explain the process of model extraction and perturbation used in GEMINI?
Answer:<s><s> [INST] <s> [INST] Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

E